In [1]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go


from sqlalchemy import create_engine

In [2]:
engine = create_engine('mssql://DESKTOP-1PS8E8C/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
connection = engine.connect()

In [3]:
# Write about the why and not the how (the how is in the code)
query_1 = """
SELECT
    SOD.SalesOrderID,
    SOH.[Status],
    SOH.ShipDate,
    FORMAT(SOH.ShipDate, 'yyyy-MM') AS SalesMonthYear,
	SOH.OnlineOrderFlag,
	SOH.SalesPersonID,
    SOH.TerritoryID,
	ST.[Group],
	SOH.TotalDue,
    SOH.SubTotal,
    SUM(SOD.OrderQty * ISNULL(PCH.StandardCost, P.StandardCost)) AS COGS,
    SUM(SOD.LineTotal) - SUM(SOD.OrderQty * ISNULL(PCH.StandardCost, P.StandardCost)) AS Margin,
    YEAR(SOH.ShipDate) AS SalesYear,
    MONTH(SOH.ShipDate) AS SalesMonth
FROM Sales.SalesOrderHeader AS SOH
INNER JOIN Sales.SalesOrderDetail AS SOD ON SOH.SalesOrderID = SOD.SalesOrderID
INNER JOIN Production.Product AS P ON p.ProductID = SOD.ProductID
LEFT JOIN Sales.SalesTerritory AS ST on st.TerritoryID = SOH.TerritoryID
LEFT JOIN Production.ProductCostHistory AS PCH on PCH.ProductID = P.ProductID and (PCH.StartDate <= SOH.ShipDate and ISNULL(PCH.enddate,'9999-12-31') > SOH.shipdate)
WHERE SOH.[Status] = 5
GROUP BY SOD.SalesOrderID, SOH.[Status], SOH.ShipDate, SOH.OnlineOrderFlag, SOH.SalesPersonID, SOH.TotalDue, SOH.SubTotal, YEAR(SOH.ShipDate), MONTH(SOH.ShipDate), SOH.TerritoryID, ST.[Group]
ORDER BY SalesYear;
"""
pd.read_sql(query_1, con=connection)

,SalesOrderID,Status,ShipDate,SalesMonthYear,OnlineOrderFlag,SalesPersonID,TerritoryID,Group,TotalDue,SubTotal,COGS,Margin,SalesYear,SalesMonth
0,44120,5,2011-08-08,2011-08,False,277.0,4,North America,30375.6923,26967.1293,24966.4890,2000.6403,2011,8
1,44087,5,2011-08-08,2011-08,False,276.0,4,North America,5641.1910,5007.7200,5046.8672,-39.1472,2011,8
2,44079,5,2011-08-08,2011-08,False,277.0,3,North America,28945.1938,25706.4831,25566.3993,140.0838,2011,8
3,43770,5,2011-06-25,2011-06,True,NaN,1,North America,3953.9884,3578.2700,2171.2942,1406.9758,2011,6
4,43700,5,2011-06-07,2011-06,True,NaN,4,North America,772.5036,699.0982,413.1463,285.9519,2011,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31460,74291,5,2014-06-11,2014-06,True,NaN,6,North America,75.6815,68.4900,25.6153,42.8747,2014,6
31461,73501,5,2014-05-28,2014-05,True,NaN,9,Pacific,612.1369,553.9700,348.8782,205.0918,2014,5
31462,63966,5,2014-01-17,2014-01,True,NaN,1,North America,132.5779,119.9800,64.6686,55.3114,2014,1
31463,73138,5,2014-05-24,2014-05,True,NaN,1,North America,8.0444,7.2800,2.7228,4.5572,2014,5


In [4]:
df1 = pd.read_sql(query_1, con=connection)

In [ ]:
df1.to_csv('output.csv', index=False)